Привет всем! 👋 Я рад поделиться с вами  решением,получил первую бронзовую медаль: https://www.kaggle.com/competitions/optiver-trading-at-the-close

В этот раз я для создания ансамбля из XGBoost и Light Gradient Boosted, максимально увеличивая количество повторного обучения. Подход включал использование 160 новых признаков, а начальные параметры были адаптированы из испытаний Optuna, похожих на те, что  можете найти в моем GitHub.

Это соревнование было невероятно увлекательным, и работа  принесла мне огромное удовольствие 😊. Я вынес массу знаний о базовых принципах, ускорении кода и сосредоточении внимания на методиках, которые обеспечивают наилучшие результаты, вместо того чтобы слишком углубляться в тренировочные данные. Огромное спасибо всем!


## установка пакетов машинного обучения для Python, используя специальные файлы установки (wheels) и  вывод

In [ ]:
%%time 

# Я импортирую модуль для очистки вывода в IPython, чтобы контролировать, что показывается в выводе ячейки
from IPython.display import clear_output

# Импортирую модуль для управления сборщиком мусора, чтобы оптимизировать использование памяти
import gc

# Устанавливаю пакеты LightGBM, XGBoost и CatBoost, используя локальные файлы .whl, полученные, например, из Kaggle Datasets
# Опция '-q' позволяет мне установить их в тихом режиме, чтобы не засорять вывод лишней информацией
!pip install /kaggle/input/lightgbm410/lightgbm-4.1.0-py3-none-manylinux_2_28_x86_64.whl -q
!pip install /kaggle/input/xgboost-2-0-0-whl/xgboost-2.0.2-py3-none-manylinux2014_x86_64.whl -q
!pip install /kaggle/input/catboost-1-2-2/catboost-1.2.2-cp310-cp310-manylinux2014_x86_64.whl -q

# После установки я очищаю вывод ячейки, чтобы сделать его более чистым и аккуратным
clear_output()

# Печатаю пустую строку, чтобы время выполнения %%time отобразилось корректно
print()


## Библиотеки

In [ ]:
%%time 

# Импортируем различные необходимые модули для работы с данными, машинным обучением и оптимизацией кода
import ctypes, gc, os, time, warnings, joblib, lightgbm as lgb, catboost, numpy as np, pandas as pd, polars as pl
from warnings import simplefilter

# Игнорируем предупреждения для более чистого вывода
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# Импортируем инструменты для работы с комбинациями, печати и моделями машинного обучения
from itertools import combinations
from pprint import pprint
from catboost import CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm
from xgboost import XGBRegressor

# Импортируем модули для оптимизации вычислений
from numba import njit, prange, jit

# Импортируем инструменты для оценки моделей и кросс-валидации
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, TimeSeriesSplit

# Импортируем инструменты для цветного вывода в консоль
from colorama import Fore, Style, init

# Очищаем вывод предыдущих команд в Jupyter notebook
clear_output()

# Выводим текущие версии используемых библиотек CatBoost и LightGBM
print(f"\nCurrent CatBoost version = {catboost.__version__}")
print(f"Current LightGBM version = {lgb.__version__}\n")

# Вызываем сборщик мусора для очистки неиспользуемой памяти
gc.collect()
print()  # Печатаем пустую строку для визуального разделения вывода


In [ ]:
%%time 

# Определяем версию и флаг вывода информации
version = 27
inference_flag = 3

# Устанавливаем различные флаги в зависимости от значения inference_flag
if inference_flag == 0:
    # Режим офлайн-тренировки без вывода дополнительной отладочной информации
    is_offline = True
    is_train = True
    is_infer = False
    debug_flag = False
    
elif inference_flag == 1:
    # Режим онлайн-тренировки с выводом отладочной информации
    is_offline = False
    is_train = True
    is_infer = True
    debug_flag = True
    
elif inference_flag == 2:
    # Режим онлайн-тренировки без вывода отладочной информации
    is_offline = False
    is_train = True
    is_infer = True
    debug_flag = False
    
elif inference_flag == 3:
    # Режим онлайн-инференса (вывода) с отладочной информацией
    is_offline = False
    is_train = False
    is_infer = True
    debug_flag = True
    
# Устанавливаем параметры для тренировки и валидации модели
max_lookback = np.nan  # Максимальный просмотр назад, значение по умолчанию не задано
split_day = 435        # День для разделения данных на тренировочные и тестовые

# Отладочные параметры, не используемые в текущей настройке
streaming = False
check_code = False
test_runtime = False

# Параметры для ансамблевых моделей и повторного обучения
use_lgb = True  # Использование модели LightGBM
use_cat = True  # Использование модели CatBoost
use_xgb = False  # Использование модели XGBoost не запланировано

# Веса для каждой модели в ансамбле
lgb_weight = 0.50
cat_weight = 0.50
xgb_weight = 0

# Параметры для повторного обучения модели
is_refit = True      # Нужно ли проводить повторное обучение
nb_refits = 12       # Количество повторных обучений
freq_refits = 6      # Частота повторных обучений
wd_refits = 450      # Временное окно для повторного обучения

# Параметры пост-обработки прогнозов
postprocess = "NA"   # Тип пост-обработки, не установлен

# Функция для пост-обработки прогнозов с целью установления суммы прогнозов равной нулю
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

# Функция для печати цветного текста в консоли с использованием библиотеки colorama
def PrintColor(text:str, color=Fore.BLUE, style=Style.BRIGHT):
    "Prints colored outputs using colorama using a text F-string"
    print(style + color + text + Style.RESET_ALL)

# Вызываем сборщик мусора для освобождения памяти
gc.collect()
print()  # Печатаем пустую строку для визуального разделения вывода


# **Детали версии**

| Version | Description | Date| Refits x Freq| Best LB Score| Post-processing|
|:-:|---|:-:|:-:|:-:|:-:|
|27 | * 170 features <br> * Alternating refits between LGBM, CB <br> * Refitting window = 470 dates <br> * 60-40 ensemble weights | 17Dec2023 | 12x6| 5.3387|NA|
|27 | * 170 features <br> * Alternating refits between LGBM, CB <br> * Refitting window = 500 dates <br> * 60-40 ensemble weights | 18Dec2023 | 12x6| |NA|
|27 | * 170 features <br> * Alternating refits between LGBM, CB <br> * Refitting window = 450 dates <br> * 50-50 ensemble weights | 18Dec2023 | 12x6| |NA|

# **Подготовка**

Обработаем временные ряды, добавляя лагированные цели для каждого 'stock_id' и 'seconds_in_bucket', но сдвигая 'date_id' на один, два и три дня вперед. После каждого добавления выполняешь очистку памяти и удаляешь строки с пропущенными значениями. Также  фильтрацию данных в зависимости от установленного флага проверки кода. Использование ctypes.CDLL("libc.so.6").malloc_trim(0) предназначено для более агрессивной очистки памяти в среде Linux.

In [ ]:
%%time 

# Читаем данные из CSV-файла в DataFrame
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")

# Создаем лагированные целевые переменные (targets) для моделирования временных зависимостей
# Здесь создается DataFrame last_targets с предыдущими значениями целей (lagged targets)

# Создаем копию некоторых колонок и переименовываем колонку с целевой переменной
last_targets = df[["stock_id", "date_id", "seconds_in_bucket", "target"]].rename(columns={"target": "prev_1_target"})
# Увеличиваем date_id на 1, чтобы сдвинуться на один день вперед
last_targets["date_id"] = last_targets["date_id"] + 1
# Объединяем оригинальный DataFrame с лагированными данными
df = df.merge(last_targets, on=["stock_id", "date_id", "seconds_in_bucket"], how="left")
# Удаляем временный DataFrame и очищаем память
del last_targets
gc.collect()
# Подсказываем системе освободить память
ctypes.CDLL("libc.so.6").malloc_trim(0)

# Аналогично добавляем вторую лагированную цель, сдвигаясь на два дня
last_targets_2 = df[["stock_id", "date_id", "seconds_in_bucket", "target"]].rename(columns={"target": "prev_2_target"})
last_targets_2["date_id"] = last_targets_2["date_id"] + 2
df = df.merge(last_targets_2, on=["stock_id", "date_id", "seconds_in_bucket"], how="left")
del last_targets_2
gc.collect()
ctypes.CDLL("libc.so.6").malloc_trim(0)

# Добавляем третью лагированную цель, сдвигаясь на три дня
last_targets_3 = df[["stock_id", "date_id", "seconds_in_bucket", "target"]].rename(columns={"target": "prev_3_target"})
last_targets_3["date_id"] = last_targets_3["date_id"] + 3
df = df.merge(last_targets_3, on=["stock_id", "date_id", "seconds_in_bucket"], how="left")
del last_targets_3
gc.collect()
ctypes.CDLL("libc.so.6").malloc_trim(0)

# Удаляем строки, где какие-либо из целей (target или лагированные) отсутствуют
df = df.dropna(subset=["target", "prev_1_target", "prev_2_target", "prev_3_target"], how="any")
# Сбрасываем индекс для чистоты данных
df.reset_index(drop=True, inplace=True)
# Выводим размер нового DataFrame
print(df.shape)

# Если установлен флаг проверки кода, ограничиваем DataFrame данными, начиная с определенного дня
if check_code:
    df = df[df.date_id >= 375].reset_index(drop=True)

# Снова вызываем сборщик мусора и подсказываем системе освободить память
gc.collect()
print()
ctypes.CDLL("libc.so.6").malloc_trim(0)


# **Добавим новые параметры**

In [ ]:
%%time 

####################################################################
# Memory reduction:-
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type to reduce memory usage.
    """;

    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col != "target":
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == "int" or str(col_type)[:4] == "uint":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    return df;

####################################################################
# RSI calculation:-
def calculate_rsi(prices, period= 14):
    rsi_values = np.zeros_like(prices);
    for col in prange(prices.shape[1]):
        price_data = prices[:, col];
        delta      = np.zeros_like(price_data);
        delta[1:]  = price_data[1:] - price_data[:-1];
        gain       = pd.Series(np.where(delta > 0, delta, 0));
        loss       = pd.Series(np.where(delta < 0, -delta, 0));
        avg_gain   = gain.rolling(window=period,
                                  min_periods=period).mean(engine='numba', engine_kwargs={"parallel": True});
        avg_loss   = loss.rolling(window=period,
                                  min_periods=period).mean(engine='numba', engine_kwargs={"parallel": True});
        rs         = avg_gain / avg_loss;
        rs         = rs.replace([np.inf, -np.inf], 1e-9);
        rsi_values[:, col] = 100 - (100 / (1 + rs));
    return rsi_values;

def generate_rsi(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"];
    sizes  = ["matched_size", "bid_size", "ask_size", "imbalance_size"];

    for stock_id, values in df.groupby(['stock_id'])[prices]:
        columns = [f'rsi_{col}' for col in values.columns];
        data    = calculate_rsi(values.values);
        df.loc[values.index, columns] = data;
    return df;

def generate_all_features(df, global_ftre: dict,
                          grouper_cols: list = ['stock_id'],
                          roll_window : list = [1, 2, 3, 5, 10],
                          ma_window   : list = [5, 10, 20],
                          ewm_window  : list = [7, 30],
                          wap_ftre_req: str = "N",
                          **kwarg
                          ):
    """
    This function generates all secondary features for the model

    Note:-
    1. We make MACD, EWM and BBbands, ASHI index for WAP only
    2. We make SMA for certain columns
    3. We use global features also
    """;

    cols      = [c for c in df.columns if c not in ["row_id", "time_id"]];
    df        = df[cols];
    prices    = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"];
    sizes     = ["matched_size", "bid_size", "ask_size", "imbalance_size"];
    roll_cols = ['matched_size', 'imbalance_size', 'reference_price',
                 'ask_price', 'bid_price', 'ask_size', 'bid_size', 'wap', 'near_price', 'far_price']
    ma_cols   = ['imbalance_size', 'reference_price', 'matched_size', 'wap'];
    ewm_cols  = ["wap"];

    df["imbalance_size"] = df["imbalance_size"] * df["imbalance_buy_sell_flag"];
    df["ratio_imb_mat"] = df["imbalance_size"] / df["matched_size"];
    df = df.drop(columns = ["imbalance_buy_sell_flag"]);

    # Date and time calculation:-
    df["seconds"] = df["seconds_in_bucket"] % 60;
    df["minute"]  = df["seconds_in_bucket"] // 60;

    # Global feature calculation:-
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict());

    # RSI calculation:-
    df = generate_rsi(df)

    # General feature calculation:-
    df["volume"]              = df.eval("ask_size + bid_size");
    df["mid_price"]           = df.eval("(ask_price + bid_price) / 2");
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)");
    df["matched_imbalance"]   = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)");
    df["size_imbalance"]      = df.eval("bid_size / ask_size");
    df['price_diff']          = df['reference_price'] - df['wap'];
    df["imbalance_momentum"]  = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size'];
    df["price_spread"]        = df["ask_price"] - df["bid_price"];
    df["spread_intensity"]    = df.groupby(['stock_id'])['price_spread'].diff();
    df['price_pressure']      = df['imbalance_size'] * (df['ask_price'] - df['bid_price']);
    df['market_urgency']      = df['price_spread'] * df['liquidity_imbalance'];
    df['depth_pressure']      = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price']);
    df['spread_depth_ratio']  = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size']);
    df['mid_price_movement']  = df.groupby(["stock_id"])['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0));
    df['micro_price']         = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size']);
    df['relative_spread']     = (df['ask_price'] - df['bid_price']) / df['wap'];
    df['high_volume']         = np.where(df['volume'] > df['global_median_size'], 1, 0);

    # Combination feature calculation:-
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})");

    # Distribution feature calculation:-
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)

    for col in roll_cols:
        for window in roll_window:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window);
            df[f"{col}_ret_{window}"]   = df.groupby('stock_id')[col].pct_change(window);

    if wap_ftre_req == "Y":
        for feature in ['imbalance_size', 'reference_price', 'matched_size', 'wap']:
            for window_size in ma_window:
                df[f'{feature}_rolling_mean_{window_size}'] = \
                df.groupby('stock_id')[feature].\
                transform(lambda x: x.rolling(window=window_size, min_periods=window_size).mean());
            for window_size in [20]:
                df[f'{feature}_rolling_std_{window_size}'] = \
                df.groupby('stock_id')[feature].\
                transform(lambda x: x.rolling(window=window_size, min_periods=window_size).std());

            # WAP feature calculation:-
            for feature in ['wap']:
                short_window, long_window = 12, 26
                for window in [short_window, long_window]:
                    df[f'{feature}_ewm_{window}'] = \
                    df.groupby('stock_id')[feature].transform(lambda x: x.ewm(span=window).mean())

            df[f'{feature}_vol_st'] = \
            df.groupby(['stock_id'])[feature].pct_change().transform(lambda x: x.rolling(window=short_window).std());
            df[f'{feature}_std_st'] = \
            df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=short_window).std());

            df[f'{feature}_macd'] = df[f'{feature}_ewm_{short_window}'] - df[f'{feature}_ewm_{long_window}'];

            # Bollinger Bands calculation:-
            df[f'{feature}_bollinger_upper'] =\
            df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=long_window).mean()) + \
            2 * df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=long_window).std());

            df[f'{feature}_bollinger_lower'] = \
            df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=long_window).mean()) - \
            2 * df.groupby(['stock_id'])[feature].transform(lambda x: x.rolling(window=long_window).std());

    return df.replace([np.inf, -np.inf], 0);

gc.collect();
print();
ctypes.CDLL("libc.so.6").malloc_trim(0);

In [ ]:
%%time 

# Если модель находится в оффлайн режиме, то разделяем данные на тренировочный и валидационный наборы
if is_offline:
    # Для тренировочного набора используем данные до определенного дня (split_day)
    df_train = df[df["date_id"] <= split_day]
    # Для валидационного набора используем данные после этого дня
    df_valid = df[df["date_id"] > split_day]
    # Выводим сообщение об оффлайн режиме и размерах наборов данных
    PrintColor("Offline mode")
    PrintColor(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    # В онлайн режиме весь набор данных используется для тренировки
    df_train = df
    PrintColor("Online mode")

# Печатаем пустую строку для визуального разделения вывода
print()

# Освобождаем память, вызывая сборщик мусора
gc.collect()

# Подсказываем операционной системе очистить неиспользуемую память
ctypes.CDLL("libc.so.6").malloc_trim(0)


In [ ]:
%%time

# Если мы находимся в режиме тренировки, вычисляем глобальные характеристики для каждого stock_id
if is_train:
    # Создаем словарь с глобальными статистическими характеристиками
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + 
                       df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + 
                    df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - 
                    df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + 
                        df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + 
                     df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - 
                     df_train.groupby("stock_id")["ask_price"].min(),
    }
    
    # В режиме оффлайн выполняем дополнительные шаги для создания и оптимизации признаков
    if is_offline:
        # Оптимизируем использование памяти тренировочного набора данных
        df_train = reduce_mem_usage(df_train)
        # Генерируем все признаки для тренировочного набора данных
        df_train_feats = generate_all_features(df_train, global_stock_id_feats)
        # Оптимизируем использование памяти сгенерированных признаков
        df_train_feats = reduce_mem_usage(df_train_feats)
        print("Build Train Feats Finished.")
        # Оптимизируем использование памяти валидационного набора данных
        df_valid = reduce_mem_usage(df_valid)
        # Генерируем все признаки для валидационного набора данных
        df_valid_feats = generate_all_features(df_valid, global_stock_id_feats)
        # Оптимизируем использование памяти сгенерированных признаков для валидации
        df_valid_feats = reduce_mem_usage(df_valid_feats)
        print("Build Valid Feats Finished.")
        
    else:
        # Для онлайн режима выполняем оптимизацию и создание признаков только для тренировочного набора данных
        df_train = reduce_mem_usage(df_train)
        df_train_feats = generate_all_features(df_train, global_stock_id_feats)
        df_train_feats = reduce_mem_usage(df_train_feats)
        print("Build Online Train Feats Finished.");

# Освобождаем память, вызывая сборщик мусора
gc.collect()
# Печатаем пустую строку для визуального разделения вывода
print()
# Подсказываем операционной системе очистить неиспользуемую память
ctypes.CDLL("libc.so.6").malloc_trim(0)


# **Тренируем**

In [ ]:
%%time 

if is_train:
    feature_name = list(df_train_feats.columns)

    lgb_params = {
        'objective': 'mae', 
        'random_state': 42,
        'device': 'gpu',
        'boosting_type': 'gbdt', 
        'learning_rate': 0.015, 
        'max_depth': 12, 
        'n_estimators': 2000 if inference_flag == 0 else 2250, 
        'num_leaves': 300, 
        'reg_alpha': 0.005, 
        'reg_lambda': 0.001, 
        'colsample_bytree': 0.6, 
        'subsample': 0.875, 
        'min_child_samples': 128,
    }
    
    
    cat_params = dict(iterations=5042 if inference_flag == 0 else 5600,
                      learning_rate=0.31464616673879614,
                      depth=9,
                      l2_leaf_reg=15.775786106845466,
                      bootstrap_type='Bernoulli',
                      subsample=0.9238669922301226,
                      loss_function='MAE',
                      eval_metric = 'MAE',
                      metric_period=1000,
                      task_type='GPU',
                      allow_writing_files=False,
                      random_state=42
                      )
    
    xgb_params = {    'tree_method'           : 'hist',
                      'device'                : "cuda",
                      'objective'             : 'reg:absoluteerror',
                      'n_estimators'          : 1800,
                      'eval_metric'           : 'mae',
                      'learning_rate'         : 0.018,
                      'max_depth'             : 11,
                      'colsample_bytree'      : 0.65,
                      'reg_alpha'             : 0.001,
                      'reg_lambda'            : 0.005,
                      'verbosity'             : 0,
                      'random_state'          : 42,
                     }
    
    
    print(f"Feature length = {len(feature_name)}")

    # infer
    df_train_target = df_train["target"]
    print("Infer Model Trainning.")
   
    if use_lgb:
        infer_lgb_params = lgb_params.copy()
        print()
        PrintColor("---> Infer LGB Params", color=Fore.MAGENTA)
        print(infer_lgb_params)
        infer_lgb_model = lgb.LGBMRegressor(**infer_lgb_params)
        infer_lgb_model.fit(df_train_feats[feature_name], df_train_target)
        joblib.dump(infer_lgb_model, f'LGB_v{version}.model');
    
    if use_cat:
        infer_cat_params = cat_params.copy()
        print()
        PrintColor("---> Infer Cat Params", color=Fore.MAGENTA)
        print(infer_cat_params)
        infer_cat_model = CatBoostRegressor(**infer_cat_params)
        infer_cat_model.fit(df_train_feats[feature_name], df_train_target)
        joblib.dump(infer_cat_model, f'CAT_v{version}.model');
        
    if use_xgb:
        infer_xgb_params = xgb_params.copy()
        print()
        PrintColor("---> Infer XGB Params", color=Fore.MAGENTA)
        print(infer_xgb_params)
        infer_xgb_model = XGBRegressor(**infer_xgb_params)
        infer_xgb_model.fit(df_train_feats[feature_name], df_train_target)
        joblib.dump(infer_xgb_model, f'XGB_v{version}.model');

    if is_offline:
        if not streaming:
            # offline predictions
            df_valid_target = df_valid["target"]

            if use_lgb:
                offline_predictions_lgb = infer_lgb_model.predict(df_valid_feats[feature_name])
            if use_cat:
                offline_predictions_cat = infer_cat_model.predict(df_valid_feats[feature_name])
            if use_xgb:
                offline_predictions_xgb = infer_xgb_model.predict(df_valid_feats[feature_name])

            offline_predictions = None
    
            if use_lgb:
                if offline_predictions is None:
                    offline_predictions = (lgb_weight * offline_predictions_lgb)
                else:
                    offline_predictions += (lgb_weight * offline_predictions_lgb)

            if use_cat:
                if offline_predictions is None:
                    offline_predictions = (cat_weight * offline_predictions_cat)
                else:
                    offline_predictions += (cat_weight * offline_predictions_cat)

            if use_xgb:
                if offline_predictions is None:
                    offline_predictions = (xgb_weight * offline_predictions_xgb)
                else:
                    offline_predictions += (xgb_weight * offline_predictions_xgb)

            zero_sum_preds = zero_sum(offline_predictions, df_valid_feats['bid_size'] + df_valid_feats['ask_size'])
            zero_mean_preds = offline_predictions - offline_predictions.mean()
            offline_score = mean_absolute_error(offline_predictions, df_valid_target)
            zero_sum_score = mean_absolute_error(zero_sum_preds, df_valid_target)
            zero_mean_score = mean_absolute_error(zero_mean_preds, df_valid_target)
            PrintColor(f"Offline Score {np.round(offline_score, 4)}", color = Fore.CYAN)
            PrintColor(f"Zero Sum Score {np.round(zero_sum_score, 4)}")
            PrintColor(f"Zero Mean Score {np.round(zero_mean_score, 4)}", color=Fore.YELLOW)
            
        else:
            # offline predictions
            offline_predictions = []
            zero_sum_preds = []
            zero_mean_preds = []
            df_valid_target = []
            qps = []
            cache = pd.DataFrame()
            counter = 0
            for dt in range(436, 481, 1):
                for t in range(0, 550, 10):

                    now_time = time.time()

                    SUBMIT_TEST = pd.read_csv(INPUT_DIR / f"stream-data/{dt}_{t}_val.csv")
                    df_valid_target_stream = SUBMIT_TEST["target"].values

                    cache = pd.concat([cache, SUBMIT_TEST], ignore_index=True, axis=0)
                    if counter > 0:
                        cache = cache.groupby(['stock_id', 'seconds_in_bucket']).tail(2).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)

                    feat = generate_all_features(cache, global_stock_id_feats)[-len(SUBMIT_TEST):]
                    feat = reduce_mem_usage(feat)

                    if use_lgb:
                        offline_predictions_lgb = infer_lgb_model.predict(feat)
                    if use_cat:
                        offline_predictions_cat = infer_cat_model.predict(feat)
                    if use_xgb:
                        offline_predictions_xgb = infer_xgb_model.predict(df_valid_feats[feature_name])

                    offline_predictions_stream = None
    
                    if use_lgb:
                        if offline_predictions_stream is None:
                            offline_predictions_stream = (lgb_weight * offline_predictions_lgb)
                        else:
                            offline_predictions_stream += (lgb_weight * offline_predictions_lgb)

                    if use_cat:
                        if offline_predictions_stream is None:
                            offline_predictions_stream = (cat_weight * offline_predictions_cat)
                        else:
                            offline_predictions_stream += (cat_weight * offline_predictions_cat)

                    if use_xgb:
                        if offline_predictions_stream is None:
                            offline_predictions_stream = (xgb_weight * offline_predictions_xgb)
                        else:
                            offline_predictions_stream += (xgb_weight * offline_predictions_xgb)

                    zero_sum_preds_stream = zero_sum(offline_predictions_stream, SUBMIT_TEST['bid_size'] + SUBMIT_TEST['ask_size'])
                    zero_mean_preds_stream = offline_predictions_stream - offline_predictions_stream.mean()

                    if counter == 1:
                        print(np.mean(zero_mean_preds_stream))

                    counter += 1

                    offline_predictions.extend(list(offline_predictions_stream))
                    zero_sum_preds.extend(list(zero_sum_preds_stream))
                    zero_mean_preds.extend(list(zero_mean_preds_stream))
                    df_valid_target.extend(list(df_valid_target_stream))

                    qps.append(time.time() - now_time)

                    if counter % 10 == 0:
                        print(cache.shape)
                        print(counter, 'qps:', np.mean(qps))

            offline_predictions = np.array(offline_predictions)
            zero_sum_preds = np.array(zero_sum_preds)
            zero_mean_preds = np.array(zero_mean_preds)
            df_valid_target = np.array(df_valid_target)

            offline_score = mean_absolute_error(offline_predictions, df_valid_target)
            zero_sum_score = mean_absolute_error(zero_sum_preds, df_valid_target)
            zero_mean_score = mean_absolute_error(zero_mean_preds, df_valid_target)
            PrintColor(f"Offline Score {np.round(offline_score, 4)}", color = Fore.CYAN)
            PrintColor(f"Zero Sum Score {np.round(zero_sum_score, 4)}")
            PrintColor(f"Zero Mean Score {np.round(zero_mean_score, 4)}", color=Fore.YELLOW)
            
ctypes.CDLL("libc.so.6").malloc_trim(0);
gc.collect();
print();


# **SUBMISSION**

In [ ]:
%%time 

# Если установлен флаг инференса (предсказания)
if inference_flag == 3:
    # Генерация глобальных признаков, аналогично первому блоку
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    };
    
    # Отбор данных для отладки на основе date_id и уменьшение использования памяти
    df_train_debug = df_train[df_train.date_id >= 470].reset_index(drop=True)
    df_train_debug = reduce_mem_usage(df_train_debug)
    # Генерация признаков для отладочного набора данных
    df_train_feats = generate_all_features(df_train_debug, global_stock_id_feats)
    df_train_feats = reduce_mem_usage(df_train_feats)
    # Удаление лишних колонок из набора признаков
    df_train_feats = df_train_feats.drop(columns=['date_id', 'time_id', "row_id", "dow", "target"], errors="ignore")
    
    # Загрузка сохраненных моделей для инференса в зависимости от выбранных алгоритмов
    
    if use_lgb:
        infer_lgb_model = joblib.load(f"/kaggle/input/optivermodels/LGBM1R_V{version}.model");
        display(infer_lgb_model);
        print();
        
    if use_cat:
        infer_cat_model = joblib.load(f"/kaggle/input/optivermodels/CBR_V{version}.model");
        display(infer_cat_model);
        print();
        
    if use_xgb:
        infer_xgb_model = joblib.load(f"/kaggle/input/optivermodels/XGBR_V{version}.model");
        display(infer_xgb_model);
        print();
        
    feature_name = list(df_train_feats.columns);
    PrintColor(f"\n\n---> Selected columns = {len(feature_name)}\n");
    with np.printoptions(linewidth = 160):
        pprint(np.array(feature_name));
    
ctypes.CDLL("libc.so.6").malloc_trim(0);
gc.collect();
print();

In [ ]:
%%time

# Если флаг is_infer установлен, начинаем процесс инференса
if is_infer:
    has_refitted = False  # Флаг, указывающий на то, было ли выполнено повторное обучение модели
    models_dict = {}  # Словарь для хранения моделей
    
    # Импортируем библиотеку optiver2023 и настраиваем среду для тестирования
    import optiver2023
    optiver2023.make_env.func_dict['__called__'] = False
    env = optiver2023.make_env()
    iter_test = env.iter_test()  # Получаем итератор для тестовых данных
    
    # Инициализируем переменные для управления процессом инференса
    counter = 0  # Счетчик для отслеживания количества итераций
    next_refit_date = None  # Следующая дата для повторного обучения
    lgb_refit_times = 0  # Количество выполненных повторных обучений модели LightGBM
    cat_refit_times = 0  # Количество выполненных повторных обучений модели CatBoost
    refit_times = 0  # Общее количество выполненных повторных обучений
    
    # Добавляем загруженные модели в словарь для удобства использования
    models_dict[f"infer_lgb_{lgb_refit_times}"] = infer_lgb_model
    models_dict[f"infer_cat_{cat_refit_times}"] = infer_cat_model
    
    # Инициализируем переменные для хранения предсказаний и метрик качества
    qps, predictions = 0, []
    
    # Инициализируем DataFrame'ы для хранения временных данных и кэша
    prev_df = pd.DataFrame()  # Для хранения данных предыдущего шага
    cache = pd.DataFrame()  # Кэш для хранения промежуточных результатов
    date_3_target = pd.DataFrame()  # Для хранения целевых значений с задержкой в 3 дня
    date_2_target = pd.DataFrame()  # Для хранения целевых значений с задержкой в 2 дня
    date_target = pd.DataFrame()  # Для хранения текущих целевых значений

# Вызываем функции для освобождения памяти
ctypes.CDLL("libc.so.6").malloc_trim(0)
gc.collect()
print()


In [ ]:
%%time 
# Если включен режим инференса
if is_infer:
    for (test, revealed_targets, sample_prediction) in iter_test:
        # Если это первая секунда в дне, обновляем цели предыдущих дней
        if test.seconds_in_bucket.iloc[0]== 0:
            date_3_target = date_2_target
            date_2_target = date_target
            date_target = revealed_targets
         # Готовим данные цели предыдущего дня    
        previous_target = \
        date_target[["stock_id", "date_id", "seconds_in_bucket", "revealed_target"]].\
        rename(columns = {"revealed_target": "prev_1_target"});
        # Пытаемся готовить данные цели предпредыдущего дня, иначе заполняем NaN
        try:
            previous_2_target = \
            date_2_target[["stock_id", "date_id", "seconds_in_bucket", "revealed_target"]].\
            rename(columns = {"revealed_target": "prev_2_target"});
            previous_2_target["date_id"] = previous_2_target["date_id"] + 1;
        except:
            previous_2_target = date_target[["stock_id", "date_id", "seconds_in_bucket"]];
            previous_2_target["prev_2_target"] = np.nan;
         # Аналогично для цели третьего дня назад   
        try:
            previous_3_target = \
            date_3_target[["stock_id", "date_id", "seconds_in_bucket", "revealed_target"]].\
            rename(columns = {"revealed_target": "prev_3_target"});
            previous_3_target["date_id"] = previous_3_target["date_id"] + 2;
        except:
            previous_3_target = date_target[["stock_id", "date_id", "seconds_in_bucket"]];
            previous_3_target["prev_3_target"] = np.nan;
            
         # Объединяем тестовый набор данных с предыдущими целями
        SUBMIT_TEST = test.merge(previous_target, on = ["stock_id", "date_id", "seconds_in_bucket"], how = "left");
        SUBMIT_TEST = SUBMIT_TEST.merge(previous_2_target, on = ["stock_id", "date_id", "seconds_in_bucket"], how = "left");
        SUBMIT_TEST = SUBMIT_TEST.merge(previous_3_target, on = ["stock_id", "date_id", "seconds_in_bucket"], how = "left");
        SUBMIT_TEST["prev_1_target"] = SUBMIT_TEST["prev_1_target"].astype('float');
        SUBMIT_TEST["prev_2_target"] = SUBMIT_TEST["prev_2_target"].astype('float');
        SUBMIT_TEST["prev_3_target"] = SUBMIT_TEST["prev_3_target"].astype('float');
        # Приведение типов данных колонок
        columns_given = ['seconds_in_bucket', 'imbalance_size',
                         'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
                         'far_price', 'near_price', 'bid_price', 'bid_size',
                         'ask_price', 'ask_size', 'wap'
                        ];
        SUBMIT_TEST[columns_given] = SUBMIT_TEST[columns_given].astype('float');
        
        if test.seconds_in_bucket.iloc[0]== 0:
            curr_date = test.date_id.iloc[0]
            try:
                curr_date = int(curr_date)
            except:
                pass
            # Если это первая секунда дня, и дата больше или равна 482, обновляем тренировочный набор данных
            if curr_date >= 482:
      
                to_concat = \
                date_target[["stock_id", "date_id", "seconds_in_bucket", "revealed_target"]].\
                rename(columns = {"revealed_target": "target"});
                to_concat["date_id"] = to_concat["date_id"] - 1;
                prev_df = \
                prev_df.merge(to_concat, on = ["stock_id", "date_id", "seconds_in_bucket"], how = "left");
                prev_df["target"] = prev_df["target"].astype('float');
                df_train = pd.concat([df_train, prev_df]).reset_index(drop=True);
                
                # Ограничиваем данные до определенной даты, чтобы предотвратить переполнение памяти
                df_train = df_train[df_train.date_id >= curr_date - wd_refits].dropna(subset=["target"]).reset_index(drop=True)

            prev_df = pd.DataFrame();
            if is_refit:
                if (next_refit_date is None and test.currently_scored.iloc[0] == True):
                    next_refit_date = curr_date;

                if curr_date == next_refit_date:
                    # Используя раскрытую цель для создания тренировочного экземпляра
                    if curr_date >= 482:
                        global_stock_id_feats = {
                            "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
                            "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
                            "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
                            "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
                            "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
                            "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
                        };
                        
                        PrintColor("\nRefitting Feature Generation in progress");
                        df_train              = reduce_mem_usage(df_train);
                        df_train_feats_refit  = generate_all_features(df_train, global_stock_id_feats);
                        df_train_feats_refit  = reduce_mem_usage(df_train_feats_refit);
                        df_train_refit_target = df_train["target"];
                        # Логика обновления тренировочных данных и повторного обучения моделей
                        if refit_times == 0 or refit_times % 2 == 0:
                            lgb_refit_times += 1;
                            lgb_refit_params = \
                            {'objective'              : 'mae', 
                             'random_state'           : 42,
                             'device'                 : 'gpu',
                             'boosting_type'          : 'gbdt', 
                             'learning_rate'          : 0.015, 
                             'max_depth'              : 12, 
                             'n_estimators'           : 2250, 
                             'num_leaves'             : 300, 
                             'reg_alpha'              : 0.005, 
                             'reg_lambda'             : 0.001, 
                             'colsample_bytree'       : 0.6, 
                             'subsample'              : 0.875, 
                             'min_child_samples'      : 128,
                             'verbose'                : -1,
                            };
                        
                            print();
                            PrintColor("Refitting in progress");
                            PrintColor("---> Refit LGB Params", color=Fore.MAGENTA);
                            print(lgb_refit_params);
                            lgb_refit_model = lgb.LGBMRegressor(**lgb_refit_params);
                            lgb_refit_model.fit(df_train_feats_refit[feature_name], df_train_refit_target);
                            PrintColor("Refitting Done!");
                            models_dict[f"infer_lgb_{lgb_refit_times}"] = lgb_refit_model;
                            print();
                        
                        if refit_times == 0 or refit_times % 2 == 1:
                            cat_refit_times += 1;
                            cat_refit_params = {'iterations'    : 5100, 
                                                'learning_rate' : 0.2774258427582013, 
                                                'depth'         : 10, 
                                                'l2_leaf_reg'   : 25.205435098066893, 
                                                'bootstrap_type': 'Bernoulli', 
                                                'subsample'     : 0.9405735849013803, 
                                                'loss_function' : 'MAE', 
                                                'eval_metric'   : 'MAE', 
                                                'metric_period' : 1000, 
                                                'task_type'     : 'GPU', 
                                                'allow_writing_files': False, 
                                                'random_state'       : 42,
                                               };

                            PrintColor("---> Refit CAT Params", color=Fore.MAGENTA);
                            print(cat_refit_params);
                            cat_refit_model = CatBoostRegressor(**cat_refit_params);
                            cat_refit_model.fit(df_train_feats_refit[feature_name], df_train_refit_target, verbose = 0);
                            PrintColor("Refitting Done!");
                            models_dict[f"infer_cat_{cat_refit_times}"] = cat_refit_model;
                            print();
                            
                        if refit_times == 0: 
                            refit_times += 2;
                        else: 
                            refit_times += 1;  
                        has_refitted = True;
                    
                    if refit_times < nb_refits: 
                        next_refit_date += freq_refits;               
         
        # Подготавливаем тестовые данные для модели и выполняем предсказания
        # Логика предсказания с использованием моделей и ансамблирования        
        prev_df = \
        pd.concat([prev_df, SUBMIT_TEST.drop(columns = ["currently_scored"])]).\
        reset_index(drop=True);
        cache = pd.concat([cache, SUBMIT_TEST], ignore_index=True, axis=0);
        
        if counter > 0:
            cache = \
            cache.groupby(['stock_id']).\
            tail(100).\
            sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True);
            
        if test.currently_scored.iloc[0]== False and (test.seconds_in_bucket.iloc[0] != 540 or test.date_id.iloc[0]!= 480):
            sample_prediction['target'] = 0;
            env.predict(sample_prediction);
            counter += 1;
            continue
            
        feat = generate_all_features(cache, global_stock_id_feats)[-len(test):];
        feat = feat[feature_name];
        
        # Collating single model predictions:-        
        lgb_prediction = None;
        if has_refitted:
            lgb_prediction = models_dict[f"infer_lgb_{lgb_refit_times}"].predict(feat);
        else:
            lgb_prediction = infer_lgb_model.predict(feat);
   
        cat_prediction = None;
        if has_refitted:
            cat_prediction = models_dict[f"infer_cat_{cat_refit_times}"].predict(feat);
        else:
            cat_prediction = infer_cat_model.predict(feat);

        # Ensembling predictions:-      
        ensemble_prediction = None
        if ensemble_prediction is None:
            ensemble_prediction = (lgb_weight * lgb_prediction);
        else:
            ensemble_prediction += (lgb_weight * lgb_prediction);

        if ensemble_prediction is None:
            ensemble_prediction = (cat_weight * cat_prediction);
        else:
            ensemble_prediction += (cat_weight * cat_prediction);

        # Post-processing ensemble predictions:-          
        if postprocess == "zero_mean":
            ensemble_prediction = ensemble_prediction - np.mean(ensemble_prediction);
        elif postprocess == "zero_sum":
            ensemble_prediction = zero_sum(ensemble_prediction, test['bid_size'] + test['ask_size']);
        else:
            pass;
          
        # Making API predictions:-           
        sample_prediction['target'] = ensemble_prediction;
        env.predict(sample_prediction);
        counter += 1;
      # Выводим предсказания после возможного повторного обучения   
    PrintColor(f"\n\nSubmission file after refits\n");
    display(sample_prediction.head(10));
    print();
    
ctypes.CDLL("libc.so.6").malloc_trim(0);
gc.collect();
print();   